In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr
from PIL import Image
from transformers import AutoTokenizer,ViTFeatureExtractor,VisionEncoderDecoderModel

In [ ]:
device='cpu'
encode_="nlpconnect/vit-gpt2-image-captioning"
decode_="nlpconnect/vit-gpt2-image-captioning"
model_="nlpconnect/vit-gpt2-image-captioning"

tokenizer=AutoTokenizer.from_pretrained(decode_)
feature_extractor=ViTFeatureExtractor.from_pretrained(encode_)
model=VisionEncoderDecoderModel.from_pretrained(model_).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.4.attn.bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.4.crossattention.masked_bias', 'decoder.transformer.h.2.crossattention.bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.6.crossattention.bias', 'decoder.transformer.h.6.attn.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.1.attn.bias', 'decoder.transformer.h.9.crossattention.masked_bias', 'decoder.transformer.h.8.attn.bias', 'decoder.transformer.h.8.crossattentio

In [ ]:
def pred(image,max_length=64,num_beams=4,num_outputs=3):
  image=image.convert('RGB')
  image=feature_extractor(image, return_tensors='pt').pixel_values.to(device)
  caption_ids=model.generate(image, max_length=max_length,num_beams=num_beams,num_return_sequences=num_outputs)
  captions=[tokenizer.decode(ids)for ids in caption_ids]
  return captions

def captioning_fn(image):
  pil_image=Image.fromarray(image.astype('uint8'),'RGB')
  captions=pred(pil_image)
  return ('Captions:',captions)

def set_example_image(example: list)->dict:
  return gr.Image.update(value=example[o])

css='''
h1{
  text-align: center;
}
h3{
  text-align:center;
}
img{
  max_width=800px;
  max_height=600px;
}
img{
  max_width=1000px;
  max_height=600px;
}'''

demo=gr.Blocks(css=css)
with demo:
  gr.Markdown('''<h1 id='title'> Image Caption Generator</h1>''')
  with gr.Column():
    input=gr.inputs.Image(type='pil',label='Upload Image',optional=True)
    output=gr.outputs.Textbox(type='text',label='Caption')
  btn=gr.Button("Generate Captions")
  btn.click(fn=pred,inputs=input,outputs=output)

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>